In [167]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns',0)
pd.set_option('display.max_colwidth',0)

import warnings
warnings.filterwarnings('ignore')

import requests
import json
import datetime as dt

In [55]:
from digital_ocean_credentials import digital_ocean_host, digital_ocean_user, digital_ocean_password, digital_ocean_port


In [152]:
import re

# 1 Connecting to the Digital Ocean Cloud SQL Database

In [2]:
!pip install mysql-connector-python

     |████████████████████████████████| 342 kB 5.0 MB/s eta 0:00:01


In [3]:
import mysql.connector

In [174]:
# Connect to the database
connection =  mysql.connector.connect(host= digital_ocean_host,
                             user= digital_ocean_user,
                             password= digital_ocean_password,
                             port = digital_ocean_port,
                             database='house_stock_watcher')

In [175]:
print(connection)

In [176]:
cursor = connection.cursor()

## --> SAMPLE QUERY

In [125]:
cursor.execute("SHOW DATABASES")
for x in cursor:
    print(x)

('defaultdb',)
('house_stock_watcher',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)


In [ ]:
#cursor.execute("SHOW TABLES")

# 2 CREATE THE TABLE - Do Not Run Again

In [50]:
create_query = "CREATE TABLE transactions (transaction_id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,disclosure_date DATE, transaction_date DATE, asset_description VARCHAR(255), ticker VARCHAR(255), type VARCHAR(255), amount VARCHAR(255), representative VARCHAR(255), district VARCHAR(255), cap_gains_over_200_usd VARCHAR(255))"

In [51]:
#cursor.execute(create_query)

In [53]:
cursor.execute("SHOW TABLES")
for x in cursor:
    print(x)

('transactions',)


# 3 LOAD THE PAST BULK CSV DATA INTO PYTHON

In [217]:
# Using index_col=[0] removes the unnamed column as the serial number / DataFrame Index is saved too 
# when saving a DataFrame

df = pd.read_csv('unclean_transactions_25_april.csv', index_col=[0])

In [218]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14175 entries, 0 to 14174
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   disclosure_date         14175 non-null  object
 1   transaction_date        14175 non-null  object
 2   asset_description       14171 non-null  object
 3   ticker                  14175 non-null  object
 4   type                    14175 non-null  object
 5   amount                  14175 non-null  object
 6   representative          14175 non-null  object
 7   district                14175 non-null  object
 8   cap_gains_over_200_usd  14175 non-null  bool  
dtypes: bool(1), object(8)
memory usage: 1010.5+ KB


## --> DATAFRAME CLEANING - BASIC

In [219]:
df.asset_description = df.asset_description.apply(lambda x: str(x))
df.asset_description = df.asset_description.apply(lambda x: re.sub("'","",x))

df.representative = df.representative.apply(lambda x: str(x))
df.representative = df.representative.apply(lambda x: re.sub("'","",x))

df['disclosure_date'] = pd.to_datetime(df['disclosure_date'],yearfirst = True, errors = 'ignore')
df['transaction_date'] = pd.to_datetime(df['transaction_date'],yearfirst = True, errors = 'ignore')
df = df.sort_values(by = 'transaction_date', ascending = False)
df.reset_index(inplace = True, drop = True)

In [220]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14175 entries, 0 to 14174
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   disclosure_date         14175 non-null  datetime64[ns]
 1   transaction_date        14175 non-null  datetime64[ns]
 2   asset_description       14175 non-null  object        
 3   ticker                  14175 non-null  object        
 4   type                    14175 non-null  object        
 5   amount                  14175 non-null  object        
 6   representative          14175 non-null  object        
 7   district                14175 non-null  object        
 8   cap_gains_over_200_usd  14175 non-null  bool          
dtypes: bool(1), datetime64[ns](2), object(6)
memory usage: 899.9+ KB


In [221]:
df.head()

,disclosure_date,transaction_date,asset_description,ticker,type,amount,representative,district,cap_gains_over_200_usd
0,2022-04-20,2022-04-19,Energy Transfer LP Common Units,ET,sale_full,"$250,001 - $500,000",Hon. Mark Dr Green,TN07,False
1,2022-04-20,2022-04-18,USA Compression partners LP Common Units Representing Limited Partner Interests,USAC,sale_full,"$500,001 - $1,000,000",Hon. Mark Dr Green,TN07,False
2,2022-04-17,2022-04-15,Microsoft Corporation - 20 Shares,MSFT,purchase,"$1,001 - $15,000",Hon. Pete Sessions,TX17,False
3,2022-04-20,2022-04-14,USA Compression partners LP Common Units Representing Limited Partner Interests,USAC,sale_full,"$100,001 - $250,000",Hon. Mark Dr Green,TN07,False
4,2022-04-20,2022-04-11,Maryland St Go St Local Facs LN BDS 5% Due 03/15/2027,--,purchase,"$1,001 - $15,000",Hon. Suzan K. DelBene,WA01,False


In [206]:
df.ticker.value_counts()

--       1137
MSFT     239 
AAPL     172 
NTAP     124 
TDDXX    122 
        ...  
ETFC     1   
BREW     1   
OWCIX    1   
OKTA     1   
BLFSD    1   
Name: ticker, Length: 2066, dtype: int64

# 4 INSERT THE DATAFRAME TO SQL

## --> Pandas DataFrame to List of TUPLES CONVERSION

In [199]:
records = df.to_records(index=False)
result = list(records)

In [111]:
# Reminder - to_records doesn't convert it into TUPLES, but rather a NumPy record which is structurally the same. (probably)
# We Still gotta convert it into Tuples
for i in result:
    print(type(i))
    break

<class 'numpy.record'>


In [185]:
len(result)

14175

In [186]:
str(result[0])

"('2022-04-20T00:00:00.000000000', '2022-04-19T00:00:00.000000000', 'Energy Transfer LP Common Units', 'ET', 'sale_full', '$250,001 - $500,000', 'Hon. Mark Dr Green', 'TN07', False)"

## --> FRAMING THE INSERT QUERY

In [200]:
insert_query = "INSERT INTO transactions (disclosure_date, transaction_date, asset_description, ticker, type,amount, representative, district, cap_gains_over_200_usd) VALUES "

In [201]:
for i in range(len(result)):
    #record = 
    insert_query+= (str(result[i]) + ",")

In [202]:
# Removing an EXTRA COMMA for the LAST ELEMENT in the string.
insert_query = insert_query[:-1]

In [203]:
insert_query

'INSERT INTO transactions (disclosure_date, transaction_date, asset_description, ticker, type,amount, representative, district, cap_gains_over_200_usd) VALUES (\'2022-04-20T00:00:00.000000000\', \'2022-04-19T00:00:00.000000000\', \'Energy Transfer LP Common Units\', \'ET\', \'sale_full\', \'$250,001 - $500,000\', \'Hon. Mark Dr Green\', \'TN07\', False),(\'2022-04-20T00:00:00.000000000\', \'2022-04-18T00:00:00.000000000\', \'USA Compression partners LP Common Units Representing Limited Partner Interests\', \'USAC\', \'sale_full\', \'$500,001 - $1,000,000\', \'Hon. Mark Dr Green\', \'TN07\', False),(\'2022-04-17T00:00:00.000000000\', \'2022-04-15T00:00:00.000000000\', \'Microsoft Corporation - 20 Shares\', \'MSFT\', \'purchase\', \'$1,001 - $15,000\', \'Hon. Pete Sessions\', \'TX17\', False),(\'2022-04-20T00:00:00.000000000\', \'2022-04-14T00:00:00.000000000\', \'USA Compression partners LP Common Units Representing Limited Partner Interests\', \'USAC\', \'sale_full\', \'$100,001 - $250

## --> Pushing the Data Manually to SQL - Do not RUN AGAIN

In [204]:
cursor.execute(insert_query)
#connection.commit()